# File Name: simple_finetunning_builder.ipynb
### Location: Chapter 10
### Purpose: 
#####             1. Download the dataset and define the training, validation, and test dataset 
#####             2. Create the customization ( fine-tuned) job
#####             3. Create provisioned throughput for the model
#####             4. Invoke the Custom ( fine-tuned) Model and test use case 
##### Dependency: simple-sageMaker-bedrock.ipynb at Chapter 3 should work properly. 
# <ins>-----------------------------------------------------------------------------------</ins>

# <ins>Amazon SageMaker Classic</ins>
#### Those who are new to Amazon SageMaker Classic. Follow the link for the details. https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html

# <ins>Environment setup of Kernel</ins>
##### Fill "Image" as "Data Science"
##### Fill "Kernel" as "Python 3"
##### Fill "Instance type" as "ml-t3-medium"
##### Fill "Start-up script" as "No Scripts"
##### Click "Select"

###### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks-create-open.html for details.

# <ins>Mandatory installation on the kernel through pip</ins>

##### This lab will work with below software version. But, if you are trying with latest version of boto3, awscli, and botocore. This code may fail. You might need to change the corresponding api. 

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell. 
##### The below installation might take arround 20-25 mins

In [ ]:
%%time 

%pip install --upgrade pip
%pip install --no-build-isolation --force-reinstall -q \
    "boto3>=1.34.84" \
    "awscli>=1.32.84" \
    "botocore>=1.34.84" \
    "langchain" \
    "typing_extensions" \
    "pypdf" \
    "urllib3" \
    "jsonlines" \
    "datasets" \
    "pandas" \
    "matplotlib" \
    "ipywidgets>=7,<8"

# <ins>Disclaimer</ins>

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell.

# <ins>Restart the kernel</ins>

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

# <ins>Python package import</ins>

##### boto3 offers various clients for Amazon Bedrock to execute various actions.
##### botocore is a low-level interface to AWS tools, while boto3 is built on top of botocore and provides additional features

In [4]:
import warnings
import json
import os
import sys
import boto3 
import time
import pprint
from datasets import load_dataset
import random
import jsonlines
import botocore
from datetime import datetime

### Ignore warning 

In [5]:
warnings.filterwarnings('ignore')

## Define important environment variable

In [ ]:
# Try-except block to handle potential errors
try:
    # Create a new Boto3 session to interact with AWS services
    boto3_session_name = boto3.session.Session()

    # Retrieve the current AWS region from the session
    aws_region_name = boto3_session_name.region_name
    
    # Create a new Boto3 bedrock client to interact with AWS services
    boto3_bedrock_client = boto3.client('bedrock')
    
    # Create a new Boto3 bedrock runtime client to interact with AWS services
    boto3_bedrock_runtime_client = boto3.client('bedrock-runtime')
    
    # Create an STS client to interact with AWS Security Token Service (STS)
    sts_client = boto3.client('sts')
    
    # Create an STS client to interact with AWS Security Token Service (STS)
    iam_client = boto3.client('iam')
    
    # Create an S3 client to interact with Amazon S3
    s3_client = boto3.client('s3')

    # Get the AWS account ID of the caller
    aws_account_id = sts_client.get_caller_identity()["Account"]
    
    # Generate a random suffix number between 200 and 900
    random_suffix = random.randrange(200, 900)
    
    # Generate a suffix using the region and account ID for the S3 bucket name
    s3_suffix = f"{aws_region_name}-{aws_account_id}"

    # Define the name of the S3 bucket (you can replace this with your actual bucket name)
    s3_bucket_name = f'bedrock-kb-{s3_suffix}-{random_suffix}'
    
    # Store all variables in a dictionary
    variables_store = {
        "boto3_session_name": boto3_session_name,
        "aws_region_name": aws_region_name,
        "boto3_bedrock_client": boto3_bedrock_client,
        "random_suffix": random_suffix,
        "boto3_bedrock_runtime_client": boto3_bedrock_runtime_client,
        "s3_suffix": s3_suffix,
        "s3_bucket_name": s3_bucket_name,
        "sts_client": sts_client,
        "aws_account_id": aws_account_id,
        "iam_client":iam_client,
        "s3_client": s3_client
    }

    # Print all variables
    for var_name, value in variables_store.items():
        print(f"{var_name}: {value}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")

### ---------------
##### The provided code snippet uses the AWS Boto3 library to manage an Amazon S3 bucket for a knowledge base data source. It begins by creating an S3 client and defines a bucket name, s3_bucket_name. A function, check_bucket_exists, checks whether the specified bucket exists by attempting to retrieve its metadata using the head_bucket method. If the bucket exists, a message is printed confirming its existence. If it does not exist (error code '404'), the function returns False. If the bucket is missing, the script proceeds to create it using the create_bucket method, ensuring the data source bucket is always available.

In [ ]:
%%time
# Check if s3 bucket exists, and if not create S3 bucket for knowledge base data source

# Try-except block to handle potential errors
try:

    # Define the bucket name (you can replace this with your actual bucket name)
    bucket_name = s3_bucket_name

    # Check if the bucket exists
    def check_bucket_exists(bucket_name):
        try:
            s3_client.head_bucket(Bucket=bucket_name)
            print(f"Bucket '{bucket_name}' already exists.")
            return True
        except botocore.exceptions.ClientError as e:
            error_code = e.response['Error']['Code']
            if error_code == '404':
                print(f"Bucket '{bucket_name}' does not exist.")
                return False
            else:
                raise e

    # If the bucket doesn't exist, create it
    if not check_bucket_exists(bucket_name):
        # Create the S3 bucket
        s3_client.create_bucket(Bucket=bucket_name)
        print(f"Bucket '{bucket_name}' created successfully.")

except botocore.exceptions.BotoCoreError as boto_error:
    print(f"An error occurred with Boto3: {boto_error}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")

# Download and prepare dataset

In [ ]:
%%time
# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print(f"Current working directory: {current_directory}")

# Construct the path to 'data/rag_use_cases' inside the current directory
data_directory = os.path.join(current_directory, 'data', 'finetunning')

# Print the resulting path
print(f"Data directory path: {data_directory}")

# Understand dataset 

### Source of the datasets: https://huggingface.co/datasets/bitext/Bitext-retail-banking-llm-chatbot-training-dataset

    Fields of the Dataset:
    --------------------------
    Each entry in the dataset comprises the following fields:

    flags: tags
    instruction: a user request from the Retail Banking domain
    category: the high-level semantic category for the intent
    intent: the specific intent corresponding to the user instruction
    response: an example of an expected response from the virtual assistant
    
    Catagory of the dataset:
    ----------------------------
    ACCOUNT: check_recent_transactions, close_account, create_account
    ATM: dispute_ATM_withdrawal, recover_swallowed_card
    CARD: activate_card, activate_card_international_usage, block_card, cancel_card, check_card_annual_fee, check_current_balance_on_card
    CONTACT: customer_service, human_agent
    FEES: check_fees
    FIND: find_ATM, find_branch
    LOAN: apply_for_loan, apply_for_mortgage, cancel_loan, cancel_mortgage, check_loan_payments, check_mortgage_payments
    PASSWORD: get_password, set_up_password
    TRANSFER: cancel_transfer, make_transfer

In [9]:
retail_data_set = load_dataset("bitext/Bitext-retail-banking-llm-chatbot-training-dataset")

In [10]:
# Instruction template
instruction_template = '''The following is an instruction detailing a task, accompanied by an input that provides additional context. Compose a response that effectively fulfills the request.

flags: {tags}
instruction: {instruction}
category: {category}
intent: {intent}
'''

### The script processes a dataset, retail_data_set, to create a JSONL file containing formatted prompts and completions based on specific fields (tags, instruction, category, intent, and response). It ensures the output directory exists, dataset access, and file writing, and logs issues such as missing keys or processing failures for individual rows. Each row is converted into a JSON object with a dynamically generated prompt (using a predefined instruction template) and a completion, which are then written line by line to the output file.

In [ ]:
%%time

# Define the directory and file path
output_file_path = os.path.join(data_directory, "retail_data.jsonl")


# Create the directory if it doesn't exist
try:
    os.makedirs(data_directory, exist_ok=True)
except Exception as e:
    print(f"Error creating directory {data_directory}: {e}")
    raise

# Assume `retail_data_set` is already loaded as a DatasetDict
try:
    train_dataset = retail_data_set['train']
except KeyError as e:
    print(f"Error accessing the dataset: {e}")
    raise

# Open the file for writing
try:
    with open(output_file_path, 'w', encoding='utf-8') as jsonl_file:
        # Loop through each row in the dataset
        for index, row in enumerate(train_dataset):
            try:
                # Use the instruction template to create the prompt
                prompt = instruction_template.format(
                    tags=row['tags'],
                    instruction=row['instruction'],
                    category=row['category'],
                    intent=row['intent']
                )
                
                # Extract the completion from the 'response' field
                completion = row['response']
                
                # Create the JSON object
                json_object = {
                    "prompt": prompt.strip(),  # Remove extra spaces if any
                    "completion": completion.strip()  # Remove extra spaces if any
                }
                
                # Write the JSON object to the JSONL file
                jsonl_file.write(json.dumps(json_object) + '\n')
            except KeyError as e:
                print(f"Missing key in row {index}: {e}")
            except Exception as e:
                print(f"Error processing row {index}: {e}")
except IOError as e:
    print(f"Error writing to file {output_file_path}: {e}")
    raise
except Exception as e:
    print(f"Unexpected error: {e}")
    raise

print(f"JSONL file has been created at {output_file_path}")

### The code snippet defines paths for training, validation, and test data files, and reads an original JSONL file into memory. It first ensures there are at least 1500 records in the dataset and raises an error if not. The data is shuffled to ensure randomness, and then split into three sets: 500 records each for training, validation, and testing. A function is defined to write the split data to separate JSONL files. Finally, the paths to the saved datasets are printed.

### Disclaimer: You can split entire datasets into 3 parts and perform finetunning. But, here, you are using 500 records to each file to save the computation cost as this is just an example to showcaseing. 
### Needs to add the limitation. 

In [ ]:
%%time

# Paths to the original and output files

train_file_name = "train_data.jsonl"
validation_file_name = "valid_data.jsonl"
test_file_name =  "test_data.jsonl"

input_file_path = output_file_path
train_file_path = os.path.join(data_directory, train_file_name)
validation_file_path = os.path.join(data_directory, validation_file_name)
test_file_path = os.path.join(data_directory, test_file_name)

# Read the original JSONL file
try:
    with open(input_file_path, 'r', encoding='utf-8') as input_file:
        data = [json.loads(line.strip()) for line in input_file.readlines()]
except IOError as e:
    print(f"Error reading the input file {input_file_path}: {e}")
    raise

# Ensure we have enough records
if len(data) < 1500:
    raise ValueError("The dataset does not contain enough records. Need at least 1500 records.")

# Shuffle the data to ensure randomness
random.shuffle(data)

# Split the data into train, validation, and test sets (500 records each)
train_data = data[:500]
valid_data = data[500:1000]
test_data = data[1000:1500]

# Function to write data to a JSONL file
def write_jsonl(file_path, data):
    try:
        with open(file_path, 'w', encoding='utf-8') as jsonl_file:
            for record in data:
                jsonl_file.write(json.dumps(record) + '\n')
    except IOError as e:
        print(f"Error writing to {file_path}: {e}")
        raise

# Write the datasets to their respective files
write_jsonl(train_file_path, train_data)
write_jsonl(validation_file_path, valid_data)
write_jsonl(test_file_path, test_data)

print(f"Training data saved to {train_file_path}")
print(f"Validation data saved to {validation_file_path}")
print(f"Testing data saved to {test_file_path}")

# Disclaimer
##### Make Sure that data_directory is pointing to the right path and data files are present. Otherwise, you need to change the above code

### The code defines a function upload_to_s3 that uploads files to an S3 bucket with error handling for common issues such as missing files or upload failures. The function first checks if the file exists locally. If the file is found, it attempts to upload the file to the S3 bucket using s3_client.upload_file.  After uploading, the function returns the S3 URI for the uploaded file. If any file upload fails, the function returns None. The S3 URIs for the uploaded files (training, validation, and test datasets) are printed if all uploads are successful.

In [ ]:
%%time

# Function to upload files to S3 with error handling
def upload_to_s3(file_name, local_path, s3_bucket, s3_key):
    try:
        # Check if the file exists locally
        if not os.path.exists(local_path):
            raise FileNotFoundError(f"File {local_path} not found.")
        
        try:
            # Attempt to upload file to S3
            s3_client.upload_file(local_path, s3_bucket, s3_key)
            print(f"Successfully uploaded {file_name} to s3://{s3_bucket}/{s3_key}")
            return f"s3://{s3_bucket}/{s3_key}"
        except boto3.exceptions.S3UploadFailedError as e:
            print(f"S3 upload failed for {file_name}: {e}")
        except boto3.exceptions.NoCredentialsError as e:
            print(f"No AWS credentials found for {file_name}: {e}")
        except Exception as e:
            print(f"Unexpected error while uploading {file_name}: {e}")

    except FileNotFoundError as e:
        print(f"File not found error for {file_name}: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
    
    return None  # Return None if upload failed


# Upload the files and store their S3 URIs in the respective variables
train_file_path_s3_uri = upload_to_s3(train_file_name, os.path.join(data_directory, train_file_name), s3_bucket_name, train_file_name)
validation_file_path_s3_uri = upload_to_s3(validation_file_name, os.path.join(data_directory, validation_file_name), s3_bucket_name, validation_file_name)
test_file_path_s3_uri = upload_to_s3(test_file_name, os.path.join(data_directory, test_file_name), s3_bucket_name, test_file_name)

# If all files are successfully uploaded, print the S3 URIs
if all([train_file_path_s3_uri, validation_file_path_s3_uri, test_file_path_s3_uri]):
    print("S3 URIs for the datasets:")
    print(f"Train URI: {train_file_path_s3_uri}")
    print(f"Validation URI: {validation_file_path_s3_uri}")
    print(f"Test URI: {test_file_path_s3_uri}")
else:
    print("One or more files failed to upload.")

# Fine out role ARN

In [ ]:
%%time

# Find out IAM role and ARN for this session

def find_iam_role_by_name_substring(substring):
    try:
        # Use list_roles to retrieve IAM roles
        response = iam_client.list_roles()

        # Filter roles by name that contains the substring
        matching_roles = [role for role in response['Roles'] if substring in role['RoleName']]

        if matching_roles:
            for role in matching_roles:
                print(f"Found Role: {role['RoleName']} | ARN: {role['Arn']}")
                genaibookedbedrocksagemakerexecutionrolearn = role['Arn']
        else:
            print(f"No roles found with name containing '{substring}'.")
            
        return genaibookedbedrocksagemakerexecutionrolearn

    except Exception as e:
        print(f"An error occurred: {str(e)}")

# Call the function with the desired substring
genaibookedbedrocksagemakerexecutionrolearn = find_iam_role_by_name_substring("GenAIBookBedrockSageMakerExecutionRole")

# Define variable for fine tunning 

In [18]:
bedrock_llm_foundation_model = "amazon.titan-text-lite-v1:0:4k"

#bedrock_llm_foundation_model = "amazon.titan-text-lite-v1"

customization_job_name_suffix = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
bedrock_model_customization_job_name = f"bedrock-model-finetune-job-{customization_job_name_suffix}"
bedrock_model_customization_model_name = f"bedrock-finetuned-model-{customization_job_name_suffix}"
# Select the customization type from "FINE_TUNING" or "CONTINUED_PRE_TRAINING". 
bedrock_model_customization_type = "FINE_TUNING"
bedrock_model_provisioned_model_name = f"bedrock-provisioned-model-{customization_job_name_suffix}"

### Amazon Titan text model customization hyperparameters:

    epochs: The number of complete passes through the training dataset. This parameter can take any integer value between 1 and 10, with a default value of 5.
    batchSize: The number of samples processed before updating the model's parameters. It can take any integer value between 1 and 64, with a default value of 1.
    learningRate: The rate at which the model's parameters are updated after each batch. This parameter can be any float value between 0.0 and 1.0, with a default value of 1.00E-5.
    learningRateWarmupSteps: The number of iterations during which the learning rate is gradually increased to the specified value. This parameter can take any integer value between 0 and 250, with a default value of 5.

##### Refer: https://docs.aws.amazon.com/bedrock/latest/userguide/cm-hp-titan-text.html

#### The code defines the hyperparameters for fine-tuning the Titan text model, including the number of epochs, batch size, and learning rate. It then specifies the S3 URIs for the training, validation (optional), and output data. The code constructs the necessary configurations for the training and validation datasets and the output location. It attempts to create a model customization job using AWS Bedrock's create_model_customization_job function, providing the required parameters such as the customization type, job name, custom model name, and role ARN. 

##### The below steps might take 80-100 mins on avg with this datasets and foundation model

In [ ]:

%%time

# Define the hyperparameters for fine-tuning Titan text model
hyper_parameters = {
    "epochCount": "2",
    "batchSize": "1",
    "learningRate": "0.00003",
}

output_file_path_s3_uri = f's3://{s3_bucket_name}/outputs/output-{bedrock_model_customization_model_name}'

# Specify your data path for training, validation (optional), and output
training_data_config = {"s3Uri": train_file_path_s3_uri}

validation_data_config = {
    "validators": [{
        "s3Uri": validation_file_path_s3_uri
    }]
}

output_data_config = {"s3Uri": output_file_path_s3_uri}

# Try to create the customization job
try:
    # Create the customization job
    training_job_response = boto3_bedrock_client.create_model_customization_job(
        customizationType=bedrock_model_customization_type,
        jobName=bedrock_model_customization_job_name,
        customModelName=bedrock_model_customization_model_name,
        roleArn=genaibookedbedrocksagemakerexecutionrolearn,
        baseModelIdentifier=bedrock_llm_foundation_model,
        hyperParameters=hyper_parameters,
        trainingDataConfig=training_data_config,
        validationDataConfig=validation_data_config,
        outputDataConfig=output_data_config
    )
    
    print("Customization job created successfully.")
    print(training_job_response)

except KeyError as e:
    print(f"Missing required parameter: {e}")
except boto3.exceptions.Boto3Error as e:
    print(f"Boto3 error occurred: {e}")
except Exception as e:
    print(f"Unexpected error occurred: {e}")

#### The code checks the status of a fine-tuning job in a loop until it is no longer "InProgress." It defines a function check_fine_tune_job_status() to fetch the job status. Initially, the job status is checked, and if it is "InProgress," the status is checked every 120 seconds. Once the job status is no longer "InProgress," the job details are retrieved, and the output job name is derived from the job ARN.

In [ ]:
%%time

# Function to check the status of the fine-tuning job
def check_fine_tune_job_status(job_name):
    try:
        # Fetch the current status of the fine-tuning job
        job_status = boto3_bedrock_client.get_model_customization_job(jobIdentifier=job_name)["status"]
        return job_status
    except KeyError as e:
        print(f"Error: Missing expected data in the response: {e}")
    except boto3.exceptions.Boto3Error as e:
        print(f"Boto3 error occurred: {e}")
    except Exception as e:
        print(f"Unexpected error occurred while fetching job status: {e}")
    return None

# Check the initial job status
fine_tune_job = check_fine_tune_job_status(bedrock_model_customization_job_name)
if fine_tune_job:
    print(f"Initial fine-tune job status: {fine_tune_job}")

    # Loop to check the status every 60 seconds until the job is no longer "InProgress"
    while fine_tune_job == "InProgress":
        print("Job is still in progress, checking again after 60 seconds...")
        time.sleep(120)  # Wait for 60 seconds before checking the status again
        fine_tune_job = check_fine_tune_job_status(bedrock_model_customization_job_name)
        if fine_tune_job:
            print(f"Current fine-tune job status: {fine_tune_job}")
    
    # Once the job is no longer "InProgress", fetch and display the final job details
    if fine_tune_job != "InProgress":
        fine_tune_job_details = boto3_bedrock_client.get_model_customization_job(jobIdentifier=bedrock_model_customization_job_name)
        print(fine_tune_job_details)  
        output_job_name = "model-customization-job-" + fine_tune_job_details['jobArn'].split('/')[-1]
        print(f"Output job name: {output_job_name}")
else:
    print("Error: Could not retrieve the fine-tuning job status.")

#### Provisioned throughput is required not only for evaluating the model's performance but also for handling custom model inferences. You specify provisioned throughput in Model Units (MU), where each model unit defines the throughput capacity for a given model. The MU determines the number of input and output tokens the model can process and generate per minute.

#### For custom models, provisioned throughput ensures that the model can handle inference requests efficiently, especially when dealing with large volumes of data or high-frequency requests. Without sufficient throughput, the model may experience delays or be unable to process requests within an acceptable timeframe.

#### Model unit quotas depend on the level of commitment to provisioned throughput. For custom models with no commitment, you are allocated one model unit per throughput, with a limit of two provisioned throughputs per account. For custom models with commitment, the default quota is 0 model units, and you may request an increase if necessary.

#### Provisioned throughput is also required after a customization job is finished, to ensure the fine-tuned model can be used effectively for inference. You can create provisioned throughput either through the AWS console or using the relevant API call. It typically takes around 20-30 minutes to complete the provisioning process.

In [ ]:
%%time

# Retrieve the custom model ARN (model identifier)
try:
    custom_model_id_arn = boto3_bedrock_client.get_custom_model(modelIdentifier=bedrock_model_customization_model_name)['modelArn']
    print(f"Custom model ARN: {custom_model_id_arn}")
except KeyError as e:
    print(f"Error: Custom model with identifier {bedrock_model_customization_model_name} not found: {e}")
except boto3.exceptions.Boto3Error as e:
    print(f"Boto3 error occurred while fetching model ARN: {e}")
except Exception as e:
    print(f"Unexpected error occurred: {e}")

# Create Provisioned Throughput
try:
    # Create provisioned throughput for the model
    provisioned_model_response = boto3_bedrock_client.create_provisioned_model_throughput(
        modelUnits=1,
        provisionedModelName=bedrock_model_provisioned_model_name,
        modelId=custom_model_id_arn
    )
    provisioned_model_id_arn = provisioned_model_response['provisionedModelArn']
    print(f"Provisioned throughput ARN: {provisioned_model_id_arn}")
    
except KeyError as e:
    print(f"Error: Failed to create provisioned throughput: {e}")
except boto3.exceptions.Boto3Error as e:
    print(f"Boto3 error occurred while creating provisioned throughput: {e}")
except Exception as e:
    print(f"Unexpected error occurred: {e}")

# Check the status of the provisioned throughput until it's completed
try:
    status_provisioning = boto3_bedrock_client.get_provisioned_model_throughput(provisionedModelId=provisioned_model_id_arn)['status']
    print(f"Provisioned throughput status: {status_provisioning}")

    while status_provisioning == 'Creating':
        time.sleep(120)  # Wait for a minute before checking the status again
        status_provisioning = boto3_bedrock_client.get_provisioned_model_throughput(provisionedModelId=provisioned_model_id_arn)['status']
        print(f"Provisioned throughput status: {status_provisioning}")
        time.sleep(120)  # Wait for another minute before the next status check

except KeyError as e:
    print(f"Error: Failed to retrieve provisioning status: {e}")
except boto3.exceptions.Boto3Error as e:
    print(f"Boto3 error occurred while checking provisioning status: {e}")
except Exception as e:
    print(f"Unexpected error occurred while checking provisioning status: {e}")

### Test the custom model with the below prompt

#### The provided code demonstrates how to invoke both a fine-tuned model and a base model using the AWS Bedrock service. First, it loads test data, extracting the prompt and reference summary for the model evaluation. The prompt is used to create the request body for invoking the models. The base model's ARN is specified, and the body includes parameters such as maxTokenCount, stopSequences, and others to control the generation. It then uses the boto3 client to invoke both the fine-tuned model and the base model. The responses are parsed, and the results from both models, along with the reference summary, are printed. 

In [25]:
tags="BCQZ"
instruction="I forgot my password. I want to reset the password."
category="PASSWORD"
intent="get_password"

prompt = instruction_template.format(
                    tags=tags,
                    instruction=instruction,
                    category=category,
                    intent=intent
                )

In [ ]:
# Define the base model ARN

bedrock_llm_foundation_model="amazon.titan-text-lite-v1"

# Define provisioned_model_arn
provisioned_model_arn=boto3_bedrock_client.list_provisioned_model_throughputs()["provisionedModelSummaries"][0]["provisionedModelArn"]


In [ ]:


    %%time
    
    # Create the body for the model invocation request
    body = json.dumps({
        "inputText": prompt,
        "textGenerationConfig": {
            "maxTokenCount": 2048,
            "stopSequences": ['User:'],
            "temperature": 0,
            "topP": 0.9
        }
    })

    # Set request headers
    accept = 'application/json'
    contentType = 'application/json'

    # Invoke the pretraining model
    pretraining_response = boto3_bedrock_runtime_client.invoke_model(
        body=body, 
        modelId=provisioned_model_arn, 
        accept=accept, 
        contentType=contentType
    )

    # Invoke the base model
    base_model_response = boto3_bedrock_runtime_client.invoke_model(
        body=body, 
        modelId=bedrock_llm_foundation_model,
        accept=accept, 
        contentType=contentType
    )

    # Parse the responses from both models
    pretraining_response_body = json.loads(pretraining_response.get('body').read())
    base_model_response_body = json.loads(base_model_response.get('body').read())
    print("Base model response: ", base_model_response_body["results"][0]["outputText"] + '\n')
    print("pretraining model response:", pretraining_response_body["results"][0]["outputText"] + '\n')

# End of NoteBook 

#### <ins>Step 1</ins> 

##### Please ensure that you close the kernel after using this notebook to avoid any potential charges to your account.

##### Process: Go to "Kernel" at top option. Choose "Shut Down Kernel". 
##### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/studio-ui.html


#### <ins>Step 2</ins> 

#### If you are not executing any further lab of this Chapter 10
##### Uncomment and execute the below code to delete the provision thoughtput and custom model

In [ ]:

'''
%%time 

# Function to delete provisioned throughput
def delete_provisioned_throughput(provisioned_model_id):
    try:
        # Attempt to delete provisioned model throughput
        print(f"Attempting to delete provisioned throughput for model ID: {provisioned_model_id}")
        response = boto3_bedrock_client.delete_provisioned_model_throughput(provisionedModelId=provisioned_model_id)
        
        # Log the response if successful
        print("Provisioned throughput deletion successful. Response:")
        print(response)
        print("\n\n")
        
    except ClientError as e:
        # Handle client error (e.g., invalid request or resource not found)
        print(f"Client error occurred while deleting provisioned throughput: {e}")
    except Exception as e:
        # Handle any other unexpected errors
        print(f"Unexpected error occurred while deleting provisioned throughput: {e}")

# Function to delete a custom model
def delete_custom_model(model_identifier):
    try:
        # Attempt to delete the custom model
        print(f"Attempting to delete custom model with identifier: {model_identifier}")
        response = boto3_bedrock_client.delete_custom_model(modelIdentifier=model_identifier)
        
        # Log the response if successful
        print("Custom model deletion successful. Response:")
        print(response)
        
    except ClientError as e:
        # Handle client error (e.g., invalid request or resource not found)
        print(f"Client error occurred while deleting custom model: {e}")
    except Exception as e:
        # Handle any other unexpected errors
        print(f"Unexpected error occurred while deleting custom model: {e}")

# Delete provisioned throughput for the custom model
delete_provisioned_throughput(provisioned_model_id_arn)

# Delete the custom model
delete_custom_model(bedrock_model_customization_model_name)
'''